# Inference
First loading librairies, dataset, transform, and model

In [3]:
from pathlib import Path
import pandas as pd
import torch
import torchaudio
import torch.nn as nn
from dataset import UrbanSoundDataset, MelSpectrogram3Channels
from model import CNN, DenseNet
from train import DATASET_DIR, SAMPLE_RATE, NUM_SAMPLES, transformation, device, BATCH_SIZE

class_mapping = [
    "air_conditioner",
    "car_horn",
    "children_playing",
    "dog_bark",
    "drilling",
    "engine_idling",
    "gun_shot",
    "jackhammer",
    "siren",
    "street_music"
]

loading weights and creating test dataset / dataloader

In [4]:

model_path = Path("./model/model_dense_DA.pth")
model = DenseNet().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()
dataset = UrbanSoundDataset(DATASET_DIR, SAMPLE_RATE, NUM_SAMPLES, transformation=transformation, device=device, folder=2, data_augment=False)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Infer on dataset

In [9]:
N = len(dataset)
targets, predictions = torch.zeros(0), torch.zeros(0)
for i, data in enumerate(dataloader):
    input, target = data
    output = model(input)
    prediction = output.argmax(axis=1)
    targets = torch.hstack([targets, target])
    predictions = torch.hstack([predictions, prediction])


KeyboardInterrupt: 

Confusion Matrix

In [15]:
targets = targets.type(torch.int32)
predictions = predictions.type(torch.int32)
from torchmetrics import ConfusionMatrix
confmat = ConfusionMatrix(num_classes=len(class_mapping))
mat = confmat(predictions, targets)  # lines = targets, columns = preds
# print(mat.diag().sum()/mat.sum() )
# mat = (mat.T / mat.sum(axis=1)).T
# print(mat.diag().sum()/mat.sum())

In [16]:
print(len(targets))
import plotly.express as px
fig=px.imshow(mat, x=class_mapping, y=class_mapping)
fig.update_layout(xaxis_title='predictions', yaxis_title='targets')
fig.show()

120
